In [5]:
# -----------------------------------------------------------
# Social Engineering Attack Prevention (Corporate Email Graph)
# -----------------------------------------------------------

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

import re
from datetime import datetime
import networkx as nx


# -----------------------------------------------------------
# Step 1: Extract date, sender, recipient information
# -----------------------------------------------------------
def get_date_from_to(Series):
    result_date = pd.Series(index=Series.index, dtype='object')
    result_from = pd.Series(index=Series.index, dtype='object')
    result_to = pd.Series(index=Series.index, dtype='object')

    for row, message in enumerate(Series):
        if row % 10000 == 0:
            print(f'Processing row {row} at {datetime.now()}...')

        if not isinstance(message, str):
            result_date[row] = np.nan
            result_from[row] = np.nan
            result_to[row] = np.nan
            continue

        message_words = message.split('\n')

        # Extract Date
        date_line = next((line for line in message_words if line.startswith('Date:')), None)
        if date_line:
            result_date[row] = date_line.replace('Date:', '').strip()
        else:
            result_date[row] = np.nan

        # Extract From
        from_line = next((line for line in message_words if line.startswith('From:')), None)
        if from_line:
            result_from[row] = re.findall(r'[\w\.-]+@[\w\.-]+\.\w+', from_line)
        else:
            result_from[row] = np.nan

        # Extract To
        to_line = next((line for line in message_words if line.startswith('To:')), None)
        if to_line:
            result_to[row] = re.findall(r'[\w\.-]+@[\w\.-]+\.\w+', to_line)
        else:
            result_to[row] = np.nan

    # Convert to datetime (safe & consistent)
    print('Converting Dates...')
    result_date = pd.to_datetime(result_date, errors='coerce', utc=True)

    # Compute timedelta since 1999-01-01
    valid_mask = result_date.notna()
    result_date_valid = result_date[valid_mask] - pd.Timestamp('1999-01-01', tz='UTC')
    result_date[valid_mask] = result_date_valid

    return result_date, result_from, result_to


# -----------------------------------------------------------
# Step 2: Generate a social graph with daily email counts
# -----------------------------------------------------------
def gen_graph_with_daily_counts(date_from_to, total_days=1448):
    G = nx.Graph()

    for index, row in date_from_to.iterrows():
        if pd.isna(row.date) or not isinstance(row.senders, list) or not isinstance(row.recipients, list):
            continue

        if isinstance(row.date, pd.Timedelta):
            day = row.date.days
            if not (0 <= day <= total_days):
                continue
        else:
            continue

        for sender in row.senders:
            for recipient in row.recipients:
                if sender == recipient:
                    continue  # Skip self-emails

                # Initialize edge if missing
                if not G.has_edge(sender, recipient):
                    G.add_edge(sender, recipient,
                               count=0,
                               daily_counts=np.zeros(total_days + 1, dtype=int))

                # Update counts
                G[sender][recipient]['count'] += 1
                G[sender][recipient]['daily_counts'][day] += 1

    return G


# -----------------------------------------------------------
# Step 3: Map nodes to integers
# -----------------------------------------------------------
def map_nodes_to_int(G):
    mapping = {node: i for i, node in enumerate(G.nodes())}
    H = nx.relabel_nodes(G, mapping)
    return H, mapping


# -----------------------------------------------------------
# Step 4: Load email data safely
# -----------------------------------------------------------
print("Loading emails.csv...")
emails = pd.read_csv('emails.csv', on_bad_lines='skip', engine='python')

emails.columns = [c.strip().lower() for c in emails.columns]
if 'file' not in emails.columns or 'message' not in emails.columns:
    raise ValueError("❌ CSV must contain 'file' and 'message' columns!")

emails['file'] = emails['file'].astype(str)
emails_nodups = emails[~emails['file'].str.contains('discussion_thread', na=False)]
emails_noautos = emails_nodups[~emails_nodups['file'].str.contains('all_documents', na=False)]
emails_noautos = emails_noautos.reset_index(drop=True)

print(f"✅ Loaded and cleaned {len(emails_noautos)} email messages.")


# -----------------------------------------------------------
# Step 5: Extract structured date/from/to data
# -----------------------------------------------------------
date_from_to = pd.DataFrame()
date_from_to['date'], date_from_to['senders'], date_from_to['recipients'] = get_date_from_to(emails_noautos['message'])

print("Initial parsed messages:", len(date_from_to))

date_from_to.dropna(subset=['date', 'senders', 'recipients'], inplace=True)
date_from_to = date_from_to[date_from_to.date >= pd.Timedelta(0)]
date_from_to = date_from_to[date_from_to.date <= pd.Timedelta(days=1448)]
print("After filtering valid entries:", len(date_from_to))


# -----------------------------------------------------------
# Step 6: Build graph with daily counts
# -----------------------------------------------------------
G_init = gen_graph_with_daily_counts(date_from_to, total_days=1448)
print(f"✅ Graph built with {G_init.number_of_nodes()} nodes and {G_init.number_of_edges()} edges.")


# -----------------------------------------------------------
# Step 7: Map nodes to integers
# -----------------------------------------------------------
H, mapping = map_nodes_to_int(G_init)
print(f"Mapped {len(mapping)} email addresses to integer IDs.")


# -----------------------------------------------------------
# Step 8: Export edge data with daily arrays
# -----------------------------------------------------------
edges_data = []
for u, v, data in H.edges(data=True):
    edges_data.append({
        'From': u,
        'To': v,
        'Count': int(data.get('count', 0)),
        'DailyCounts': data['daily_counts'].tolist()  # convert numpy array to list for CSV
    })

edges_df = pd.DataFrame(edges_data)
edges_df.to_csv('edges_with_daily_counts.csv', index=False)

print("✅ edges_with_daily_counts.csv saved successfully!")
print(edges_df.head())


Loading emails.csv...
✅ Loaded and cleaned 617724 email messages.
Processing row 0 at 2025-10-10 15:46:32.346222...
Processing row 10000 at 2025-10-10 15:46:32.467526...
Processing row 20000 at 2025-10-10 15:46:32.566483...
Processing row 30000 at 2025-10-10 15:46:32.676205...
Processing row 40000 at 2025-10-10 15:46:32.787565...
Processing row 50000 at 2025-10-10 15:46:32.866333...
Processing row 60000 at 2025-10-10 15:46:32.926701...
Processing row 70000 at 2025-10-10 15:46:32.987511...
Processing row 80000 at 2025-10-10 15:46:33.078712...
Processing row 90000 at 2025-10-10 15:46:33.167462...
Processing row 100000 at 2025-10-10 15:46:33.534846...
Processing row 110000 at 2025-10-10 15:46:33.632005...
Processing row 120000 at 2025-10-10 15:46:33.736597...
Processing row 130000 at 2025-10-10 15:46:33.883837...
Processing row 140000 at 2025-10-10 15:46:33.984787...
Processing row 150000 at 2025-10-10 15:46:34.053941...
Processing row 160000 at 2025-10-10 15:46:34.127214...
Processing ro

/tmp/ipykernel_78950/3449478456.py:60: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  result_date = pd.to_datetime(result_date, errors='coerce', utc=True)
/tmp/ipykernel_78950/3449478456.py:65: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<TimedeltaArray>
['864 days 23:39:00', '854 days 20:51:00', '656 days 10:00:00',
 '661 days 13:13:00', '608 days 12:07:00', '608 days 11:17:00',
 '599 days 14:44:00', '560 days 13:59:00', '655 days 09:26:00',
 '654 days 13:44:00',
 ...
 '656 days 11:12:00', '657 days 20:25:00', '657 days 20:30:00',
 '620 days 15:44:00', '658 days 12:14:00', '661 days 10:28:00',
 '662 days 10:55:00', '663 days 09:21:00', '663 days 14:09:00',
 '663 days 23:55:00']
Length: 617724, dtype: timedelta64[ns]' has dtype incompatible with datetime64[ns, UTC],

Initial parsed messages: 617724
After filtering valid entries: 281416
✅ Graph built with 32609 nodes and 79118 edges.
Mapped 32609 email addresses to integer IDs.
✅ edges_with_daily_counts.csv saved successfully!
   From  To  Count                                        DailyCounts
0     0   1     19  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1     0   2     66  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2     0   3      2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3     0   4     14  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4     0   5      4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
